# Sequence Model Research

The scope of this notebook is to assess and train different sequence models given the training data generated.

Training data is generated based on financial time series data labeled with potential profits using a buy-sell system.

The goal is to create a sequence model that can choose favourable stock charts equal to or better than a human can via traditional technical analysis.

# Import Libraries and Data

In [1]:
import os
import numpy as np

# Define the data directory relative to the script location
data_dir = 'data'

# Define the file paths
sequences_path = os.path.join(data_dir, 'sequences.npy')
labels_path = os.path.join(data_dir, 'labels.npy')
metadata_path = os.path.join(data_dir, 'metadata.npy')

# List of feature names
feature_names = [
    'Consol_Len_Bars', 'Consol_Depth_Percent',
    'Distance_to_21EMA', 'Distance_to_50SMA', 'Distance_to_200SMA', 
    'RSL_NH_Count', 'RSL_Slope', 'Up_Down_Days', 
    'Stage 2', 'UpDownVolumeRatio', 'ATR', '%B'
]

# Load the data
try:
    data_sequences = np.load(sequences_path)
    data_labels = np.load(labels_path)
    data_metadata = np.load(metadata_path)

    # Number of examples to select
    num_examples = 115000

    # Generate a random permutation of indices
    indices = np.random.permutation(len(data_sequences))

    # Select the first `num_examples` indices
    selected_indices = indices[:num_examples]

    # Use the selected indices to create the random subset
    data_sequences = data_sequences[selected_indices, :, :]
    data_labels = data_labels[selected_indices]
    data_metadata = data_metadata[selected_indices]

    # Inspect the shape and size of the loaded data before slicing
    print(f'Loaded sequences shape: {data_sequences.shape}')
    print(f'Loaded sequences size: {data_sequences.size}')
    print(f'Loaded labels shape: {data_labels.shape}')
    print(f'Loaded metadata shape: {data_metadata.shape}')

except FileNotFoundError as e:
    print(f"Error loading files: {e}")
except ValueError as e:
    print(f"Value error: {e}")


Loaded sequences shape: (115000, 63, 12)
Loaded sequences size: 86940000
Loaded labels shape: (115000,)
Loaded metadata shape: (115000, 2)


## Data Preprocessing

### NaN anf INF Removal

In [2]:
import numpy as np

# Dictionary to map variable names to their corresponding data arrays
data_dict = {
    'data_sequences': data_sequences,
    'data_labels': data_labels,
}

# Using a dictionary to iterate over variables
for var_name, data in data_dict.items():
    num_nans = np.sum(np.isnan(data))
    num_infs = np.sum(np.isinf(data))
    print(f"NaNs in {var_name}: {num_nans}")
    print(f"Infs in {var_name}: {num_infs}")

    # Remove NaNs and Infs
    if num_nans > 0 or num_infs > 0:
        data_dict[var_name][:] = np.nan_to_num(data, nan=0.0, posinf=0.0, neginf=0.0)
        num_nans_after = np.sum(np.isnan(data))
        num_infs_after = np.sum(np.isinf(data))
        print(f"NaNs remaining in {var_name} after removal: {num_nans_after}")
        print(f"Infs remaining in {var_name} after removal: {num_infs_after}")

print("NaN and Inf removal completed.")

NaNs in data_sequences: 904983
Infs in data_sequences: 33
NaNs remaining in data_sequences after removal: 0
Infs remaining in data_sequences after removal: 0
NaNs in data_labels: 0
Infs in data_labels: 0
NaN and Inf removal completed.


### Corrupted sequence removal

99% of stocks I buy will be below 1000, with a few above 1000, although they are important.

I also noticed quite a few training examples have weird price data, which I filter out below.

I noticed with thresholds above 3e3, the max is the threshold, which is very suspect.

The loss of training examples is insignificant, and the result is better normalization of the data and obviously no corrupted sequences.

#### Feature Stats

In [3]:
import numpy as np
import pandas as pd

def print_feature_stats(data_sequences, feature_names):
    print("Feature Statistics:")
    print("-" * 50)

    for i, feature_name in enumerate(feature_names):
        feature_data = data_sequences[:, :, i].flatten()
        
        stats = {
            "Mean": np.mean(feature_data),
            "Median": np.median(feature_data),
            "Std Dev": np.std(feature_data),
            "Min": np.min(feature_data),
            "Max": np.max(feature_data),
            "25th Percentile": np.percentile(feature_data, 25),
            "75th Percentile": np.percentile(feature_data, 75),
            "Skewness": pd.Series(feature_data).skew(),
            "Kurtosis": pd.Series(feature_data).kurtosis(),
            "Zero Count": np.sum(feature_data == 0),
            "Zero Percentage": np.mean(feature_data == 0) * 100
        }
        
        print(f"Feature: {feature_name}")
        for stat_name, stat_value in stats.items():
            print(f"  {stat_name}: {stat_value:.4f}")
        print("-" * 50)

# Call the function to print statistics
print_feature_stats(data_sequences, feature_names)

# Additional overall statistics
print("Overall Dataset Statistics:")
print(f"Total number of sequences: {data_sequences.shape[0]}")
print(f"Sequence length: {data_sequences.shape[1]}")
print(f"Number of features: {data_sequences.shape[2]}")
print(f"Total number of data points: {data_sequences.size}")
print(f"Memory usage: {data_sequences.nbytes / (1024 * 1024):.2f} MB")

Feature Statistics:
--------------------------------------------------
Feature: Consol_Len_Bars
  Mean: 69.4729
  Median: 27.0000
  Std Dev: 121.2268
  Min: 0.0000
  Max: 3142.0000
  25th Percentile: 6.0000
  75th Percentile: 77.0000
  Skewness: 4.4634
  Kurtosis: 37.2080
  Zero Count: 1416240.0000
  Zero Percentage: 19.5478
--------------------------------------------------
Feature: Consol_Depth_Percent
  Mean: 16.4673
  Median: 15.6804
  Std Dev: 11.7338
  Min: 0.0000
  Max: 54.4286
  25th Percentile: 7.3934
  75th Percentile: 25.9320
  Skewness: 0.2050
  Kurtosis: -0.8840
  Zero Count: 1340533.0000
  Zero Percentage: 18.5029
--------------------------------------------------
Feature: Distance_to_21EMA
  Mean: 0.8504
  Median: 0.8903
  Std Dev: 5.5301
  Min: -99.9886
  Max: 712.1856
  25th Percentile: -1.2132
  75th Percentile: 3.0191
  Skewness: 1.0681
  Kurtosis: 95.0132
  Zero Count: 17874.0000
  Zero Percentage: 0.2467
--------------------------------------------------
Feature: D

### Normalization of Training Data

In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Define the indices based on the provided feature names
feature_names = [
    'Consol_Len_Bars', 'Consol_Depth_Percent',
    'Distance_to_21EMA', 'Distance_to_50SMA', 'Distance_to_200SMA', 
    'RSL_NH_Count', 'RSL_Slope', 'Up_Down_Days', 
    'Stage 2', 'UpDownVolumeRatio', 'ATR', '%B'
]

feature_indices = {name: idx for idx, name in enumerate(feature_names)}

# Function to remove outliers and cap values
def preprocess_data(sequences, labels):
    # Reshape sequences to 2D array for easier processing (flatten the timesteps)
    num_sequences, num_timesteps, num_features = sequences.shape
    sequences_reshaped = sequences.reshape(-1, num_features)
    
    # Create a mask to filter out invalid sequences
    valid_mask = (
        (sequences_reshaped[:, feature_indices['Distance_to_21EMA']] <= 100) &
        (sequences_reshaped[:, feature_indices['Distance_to_50SMA']] <= 200) &
        (sequences_reshaped[:, feature_indices['Distance_to_200SMA']] <= 500)
    )
    
    # Reshape the valid_mask to match the original sequence shape
    valid_mask_reshaped = valid_mask.reshape(num_sequences, num_timesteps)
    
    # Filter out sequences with any invalid timesteps
    valid_sequences_mask = valid_mask_reshaped.all(axis=1)
    filtered_sequences = sequences[valid_sequences_mask]
    filtered_labels = labels[valid_sequences_mask]
    
    # Cap 'UpDownVolumeRatio' at 10
    filtered_sequences[:, :, feature_indices['UpDownVolumeRatio']] = np.minimum(
        filtered_sequences[:, :, feature_indices['UpDownVolumeRatio']], 10
    )
    
    # Normalize the features using Z-score normalization
    scaler = StandardScaler()
    
    # Flatten the sequence again for normalization
    filtered_sequences_reshaped = filtered_sequences.reshape(-1, num_features)
    
    # Normalize
    normalized_data_reshaped = scaler.fit_transform(filtered_sequences_reshaped)
    
    # Reshape back to the original 3D shape
    normalized_data = normalized_data_reshaped.reshape(filtered_sequences.shape)
    
    return normalized_data, filtered_labels

# Function to print feature statistics
def print_feature_stats(data_sequences, feature_names):
    print("Feature Statistics:")
    print("-" * 50)

    for i, feature_name in enumerate(feature_names):
        feature_data = data_sequences[:, :, i].flatten()
        
        stats = {
            "Mean": np.mean(feature_data),
            "Median": np.median(feature_data),
            "Std Dev": np.std(feature_data),
            "Min": np.min(feature_data),
            "Max": np.max(feature_data),
            "25th Percentile": np.percentile(feature_data, 25),
            "75th Percentile": np.percentile(feature_data, 75),
            "Skewness": pd.Series(feature_data).skew(),
            "Kurtosis": pd.Series(feature_data).kurtosis(),
            "Zero Count": np.sum(feature_data == 0),
            "Zero Percentage": np.mean(feature_data == 0) * 100
        }
        
        print(f"Feature: {feature_name}")
        for stat_name, stat_value in stats.items():
            print(f"  {stat_name}: {stat_value:.4f}")
        print("-" * 50)

# Example usage with data_sequences and data_labels
# Assuming data_sequences is loaded and has shape (115000, 63, 12)

# Process the data
normalized_data, processed_labels = preprocess_data(data_sequences, data_labels)




#### Stats again

In [5]:
# Print statistics
print_feature_stats(normalized_data, feature_names)

# Additional overall statistics
print("Overall Dataset Statistics:")
print(f"Total number of sequences: {data_sequences.shape[0]}")
print(f"Sequence length: {data_sequences.shape[1]}")
print(f"Number of features: {data_sequences.shape[2]}")
print(f"Total number of data points: {data_sequences.size}")
print(f"Memory usage: {data_sequences.nbytes / (1024 * 1024):.2f} MB")

Feature Statistics:
--------------------------------------------------
Feature: Consol_Len_Bars
  Mean: 0.0000
  Median: -0.3506
  Std Dev: 1.0000
  Min: -0.5733
  Max: 25.3382
  25th Percentile: -0.5238
  75th Percentile: 0.0617
  Skewness: 4.4621
  Kurtosis: 37.1883
  Zero Count: 0.0000
  Zero Percentage: 0.0000
--------------------------------------------------
Feature: Consol_Depth_Percent
  Mean: 0.0000
  Median: -0.0671
  Std Dev: 1.0000
  Min: -1.4040
  Max: 3.2357
  25th Percentile: -0.7730
  75th Percentile: 0.8062
  Skewness: 0.2051
  Kurtosis: -0.8829
  Zero Count: 0.0000
  Zero Percentage: 0.0000
--------------------------------------------------
Feature: Distance_to_21EMA
  Mean: -0.0000
  Median: 0.0091
  Std Dev: 1.0000
  Min: -18.5588
  Max: 18.0986
  25th Percentile: -0.3779
  75th Percentile: 0.4006
  Skewness: -0.1968
  Kurtosis: 14.0993
  Zero Count: 0.0000
  Zero Percentage: 0.0000
--------------------------------------------------
Feature: Distance_to_50SMA
  Mean

# Model Definition

## Model Class

## HP Search

In [6]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns
import copy
import time
import optuna
import json
import logging
import mlflow
import mlflow.pytorch

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set random seed for reproducibility
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

# Assuming normalized_data and processed_labels are defined
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    normalized_data, processed_labels, test_size=0.2, random_state=RANDOM_SEED
)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.FloatTensor(y_train).to(device)
X_test_tensor = torch.FloatTensor(X_test).to(device)
y_test_tensor = torch.FloatTensor(y_test).to(device)

# Define the model architecture (same as before)
class StockAnalysisModel(nn.Module):
    # ... (same as before)

def save_checkpoint(epoch, model, optimizer, best_f1, params, checkpoint_path):
    # ... (same as before)

def load_checkpoint(checkpoint_path, model, optimizer):
    # ... (same as before)

def objective(trial):
    # Hyperparameter search space
    params = {
        'PROFIT_THRESH': trial.suggest_float('PROFIT_THRESH', 0.0, 0.1),
        'LEARNING_RATE': trial.suggest_float('LEARNING_RATE', 1e-5, 1e-2, log=True),
        'DROPOUT_RATE_LSTM': trial.suggest_float('DROPOUT_RATE_LSTM', 0.1, 0.5),
        'DROPOUT_RATE_LINEAR': trial.suggest_float('DROPOUT_RATE_LINEAR', 0.1, 0.5),
        'L2_REGULARIZATION': trial.suggest_float('L2_REGULARIZATION', 1e-6, 1e-3, log=True),
        'PREDICTION_THRESHOLD': trial.suggest_float('PREDICTION_THRESHOLD', 0.1, 0.9),
        'LSTM_LAYERS': trial.suggest_categorical('LSTM_LAYERS', ['64', '128', '64,32', '128,64', '128,64,32']),
        'LINEAR_LAYERS': trial.suggest_categorical('LINEAR_LAYERS', ['32', '64', '32,16', '64,32', '64,32,16']),
        'BATCH_SIZE': trial.suggest_categorical('BATCH_SIZE', [32, 64, 128, 256]),
        'SKIP_CONNECTIONS': trial.suggest_categorical('SKIP_CONNECTIONS', [False, True])
    }
    
    # Set up MLflow
    mlflow.set_experiment("Stock Analysis Model Tuning")
    with mlflow.start_run():
        # Log hyperparameters
        mlflow.log_params(params)
        
        # Prepare data
        y_train_binary = (y_train > params['PROFIT_THRESH']).astype(int)
        y_test_binary = (y_test > params['PROFIT_THRESH']).astype(int)
        
        # Log class distribution
        train_class_distribution = np.bincount(y_train_binary)
        test_class_distribution = np.bincount(y_test_binary)
        logger.info(f"Train class distribution: {train_class_distribution}")
        logger.info(f"Test class distribution: {test_class_distribution}")
        train_positive_class_percentage = train_class_distribution[1] / len(y_train_binary) * 100
        test_positive_class_percentage = test_class_distribution[1] / len(y_test_binary) * 100
        mlflow.log_metric("train_positive_class_percentage", train_positive_class_percentage)
        mlflow.log_metric("test_positive_class_percentage", test_positive_class_percentage)
        logger.info(f"Train positive class percentage: {train_positive_class_percentage:.2f}%")
        logger.info(f"Test positive class percentage: {test_positive_class_percentage:.2f}%")
        
        # Compute class weights
        class_weights = compute_class_weight('balanced', classes=np.unique(y_train_binary), y=y_train_binary)
        class_weights = torch.FloatTensor(class_weights).to(device)
        logger.info(f"Class weights: {class_weights}")
        
        train_dataset = TensorDataset(X_train_tensor, torch.FloatTensor(y_train_binary).to(device))
        train_loader = DataLoader(train_dataset, batch_size=params['BATCH_SIZE'], shuffle=True)
        
        # Initialize model
        model = StockAnalysisModel(
            input_size=X_train.shape[2],
            lstm_layers=params['LSTM_LAYERS'],
            linear_layers=params['LINEAR_LAYERS'],
            dropout_rate_lstm=params['DROPOUT_RATE_LSTM'],
            dropout_rate_linear=params['DROPOUT_RATE_LINEAR'],
            skip_connections=params['SKIP_CONNECTIONS']
        ).to(device)
        logger.info(f"Model architecture: {model}")
        
        # Define loss function and optimizer
        criterion = nn.BCELoss(reduction='none')
        optimizer = torch.optim.Adam(model.parameters(), lr=params['LEARNING_RATE'], weight_decay=params['L2_REGULARIZATION'])
        
        # Directory to save checkpoints
        checkpoint_dir = './checkpoints'
        os.makedirs(checkpoint_dir, exist_ok=True)
        checkpoint_path = os.path.join(checkpoint_dir, 'model_checkpoint.pth')
        start_epoch = 0
        best_f1 = 0
        previous_params = None

        # Load checkpoint if exists
        if os.path.exists(checkpoint_path):
            try:
                start_epoch, best_f1, previous_params = load_checkpoint(checkpoint_path, model, optimizer)
                logger.info(f"Resuming training from epoch {start_epoch} with best F1 score {best_f1:.4f}")
            except Exception as e:
                logger.warning(f"Could not load checkpoint: {e}. Starting from scratch.")

        # Ensure model architecture matches
        if previous_params and params != previous_params:
            logger.info(f"Model architecture changed, not loading from checkpoint. Previous params: {previous_params}, Current params: {params}")
            start_epoch = 0
            best_f1 = 0

        # Training loop
        n_epochs = 50
        patience = 10
        no_improve_count = 0
        for epoch in range(start_epoch, n_epochs):
            model.train()
            total_loss = 0
            all_train_preds = []
            all_train_labels = []
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                try:
                    outputs = model(batch_X)
                    # Apply class weights manually
                    weights = class_weights[batch_y.long()]
                    losses = criterion(outputs, batch_y)
                    weighted_losses = losses * weights
                    loss = weighted_losses.mean()
                    loss.backward()
                    optimizer.step()
                    total_loss += loss.item()
                    
                    all_train_preds.extend(outputs.detach().cpu().numpy())
                    all_train_labels.extend(batch_y.cpu().numpy())
                except RuntimeError as e:
                    logger.error(f"Error in forward pass: {str(e)}")
                    logger.error(f"batch_X shape: {batch_X.shape}, batch_y shape: {batch_y.shape}")
                    return 0  # Return 0 to indicate failure to Optuna
            
            # Calculate training metrics
            train_preds_binary = (np.array(all_train_preds) > params['PREDICTION_THRESHOLD']).astype(int)
            train_precision = precision_score(all_train_labels, train_preds_binary, zero_division=0)
            train_recall = recall_score(all_train_labels, train_preds_binary, zero_division=0)
            train_f1 = f1_score(all_train_labels, train_preds_binary, zero_division=0)
            
            # Validation
            model.eval()
            with torch.no_grad():
                y_pred = model(X_test_tensor)
                y_pred_binary = (y_pred > params['PREDICTION_THRESHOLD']).float()
                
                # Calculate metrics
                precision = precision_score(y_test_binary, y_pred_binary.cpu().numpy(), zero_division=0)
                recall = recall_score(y_test_binary, y_pred_binary.cpu().numpy(), zero_division=0)
                f1 = f1_score(y_test_binary, y_pred_binary.cpu().numpy(), zero_division=0)
                
                logger.info(f"Epoch {epoch}: Loss: {total_loss/len(train_loader):.4f}")
                logger.info(f"Train - Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}")
                logger.info(f"Val - Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
                
                if f1 > best_f1:
                    best_f1 = f1
                    best_model = copy.deepcopy(model)
                    no_improve_count = 0
                else:
                    no_improve_count += 1
            
            # Save checkpoint
            save_checkpoint(epoch + 1, model, optimizer, best_f1, params, checkpoint_path)
            
            # Log metrics
            mlflow.log_metric("train_loss", total_loss / len(train_loader), step=epoch)
            mlflow.log_metric("train_precision", train_precision, step=epoch)
            mlflow.log_metric("train_recall", train_recall, step=epoch)
            mlflow.log_metric("train_f1", train_f1, step=epoch)
            mlflow.log_metric("val_precision", precision, step=epoch)
            mlflow.log_metric("val_recall", recall, step=epoch)
            mlflow.log_metric("val_f1", f1, step=epoch)

            # Early stopping
            if no_improve_count >= patience:
                logger.info(f"Early stopping triggered. No improvement for {patience} epochs.")
                break
        
        # Final evaluation
        best_model.eval()
        with torch.no_grad():
            y_pred = best_model(X_test_tensor)
            y_pred_binary = (y_pred > params['PREDICTION_THRESHOLD']).float()
            
            precision = precision_score(y_test_binary, y_pred_binary.cpu().numpy(), zero_division=0)
            recall = recall_score(y_test_binary, y_pred_binary.cpu().numpy(), zero_division=0)
            f1 = f1_score(y_test_binary, y_pred_binary.cpu().numpy(), zero_division=0)
            conf_matrix = confusion_matrix(y_test_binary, y_pred_binary.cpu().numpy())
            
            # Calculate the percentage of positive predictions
            positive_pred_percentage = y_pred_binary.mean().item() * 100
            
            # Log final metrics
            mlflow.log_metric("test_precision", precision)
            mlflow.log_metric("test_recall", recall)
            mlflow.log_metric("test_f1", f1)
            mlflow.log_metric("positive_pred_percentage", positive_pred_percentage)
            
            logger.info(f"Test Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
            logger.info(f"Positive prediction percentage: {positive_pred_percentage:.2f}%")
            logger.info(f"Confusion Matrix:\n{conf_matrix}")
            
            # Log confusion matrix as a figure
            plt.figure(figsize=(10,8))
            sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
            plt.title('Confusion Matrix')
            plt.ylabel('True label')
            plt.xlabel('Predicted label')
            plt.savefig("confusion_matrix.png")
            mlflow.log_artifact("confusion_matrix.png")
            
            # Log the PyTorch model
            mlflow.pytorch.log_model(best_model, "model")
        
        # Penalize if the model is just predicting the majority class
        if positive_pred_percentage < 0.1 or positive_pred_percentage > 99.9:
            f1 = 0
        
        logger.info(f"Trial completed. Best F1 score: {f1:.4f}")
        return f1

# Run Optuna optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print and log the best parameters and score
best_params = study.best_params
best_score = study.best_value
logger.info(f"Best parameters: {best_params}")
logger.info(f"Best precision score: {best_score}")

# Save the best parameters to a JSON file
with open('best_params.json', 'w') as f:
    json.dump(best_params, f)

logger.info("Hyperparameter tuning completed. Best parameters saved to 'best_params.json'.")

[I 2024-06-29 12:36:50,815] A new study created in memory with name: no-name-bb33d8e3-bc2b-4ebf-89d5-c2b84302c01d
2024/06/29 12:48:57 WARNING mlflow.utils.requirements_utils: Found torch version (2.3.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.3.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/06/29 12:49:02 WARNING mlflow.utils.requirements_utils: Found torch version (2.3.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.3.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
[I 2024-06-29 12:49:03,033] Trial 0 finished with value: 0.1447596988998263 and parameters: {'PROFIT_THRESH': 0.023150876110827015, 'L